# Counties coordinates (lat, lon)
**Runs in about 30 s**

Two files are required to execute this code
- The top 500 cities in the US by population, based on 2010 census. Can be found at:
https://chronicdata.cdc.gov/500-Cities-Places/500-Cities-City-level-Data-GIS-Friendly-Format-201/pf7q-w24q

- All the US counties with the geolocation. Can be found at: https://www.weather.gov/gis/counties

In [1]:
# Import base packages
import pandas as pd
import numpy as np
import shapefile
import geopy.distance as gd
from fastkml import kml

Package `lxml` missing. Pretty print will be disabled


In [2]:
# Specify absolute path (this is where you stored the downloaded files above)
coords_dir = "/Users/alessandropreviero/Desktop/vaccine-allocation/notebooks/counties_geo_clean.csv"
top_cities_dir = "/Users/alessandropreviero/Downloads/top500.csv"

### Read top cities

In [6]:
valid_cols = ['StateAbbr', 'PlaceName', 'Population2010', 'Geolocation']
cities_df = pd.read_csv(top_cities_dir, usecols=valid_cols).sort_values('Population2010', ascending=False, )

cities_df.rename(columns={
    'StateAbbr': 'state', 
    'PlaceName': 'city', 
    'Population2010': 'population', 
    'Geolocation': 'geolocation'
}, inplace=True)

# originally tuple of geolocation is a string, convert to numerical
cities_df['geolocation'] = [eval(v) for v in cities_df['geolocation']]

cities_df.reset_index(drop=True, inplace=True)
cities_df.head()

,state,city,population,geolocation
0,NY,New York,8175133,"(40.694960689, -73.9313850409)"
1,CA,Los Angeles,3792621,"(34.1182277898, -118.408500088)"
2,IL,Chicago,2695598,"(41.8372950615, -87.6862308732)"
3,TX,Houston,2099451,"(29.7806691396, -95.3860033966)"
4,PA,Philadelphia,1526006,"(40.0093147808, -75.1333888571)"


# SKIP ALL THIS

### Sub sample a portion of these cities using greedy approach

In [28]:
# For each state, find which ones have more than 5 cities
cities_per_state = cities_df.state.value_counts()
excess_states = cities_per_state[cities_per_state > 5].index.to_list()

In [29]:
# Put aside states with less than 5 cities
states_few_cities = cities_per_state[cities_per_state <= 5].index.to_list()
few_cities_df = cities_df[cities_df.state.isin(states_few_cities)]

In [30]:
# Greedy approach: pick most populus city in each state, compute its distance to other cities within state
# Then pick the furthest city from it, and randomly sample 40% of remaining cities.

# Keep track of the indices of cities in the ORIGINAL cities_df
all_indices = []

for st in excess_states:
    
    state_cities_df = cities_df[cities_df.state == st]
    top_city = state_cities_df.iloc[0]
    top_city_idx = state_cities_df.index[0]
    all_indices.append(top_city_idx)
    
    dists_from_top_city = []
    
    # Skip top city
    for idx in range(1, state_cities_df.shape[0]):
        other_city = state_cities_df.iloc[idx]
        dists_from_top_city.append(gd.distance(top_city['geolocation'], other_city['geolocation']).km)
    
    # Pick furthest city
    furthest_city = state_cities_df.iloc[np.argmax(dists_from_top_city) + 1]
    furthest_city_idx = state_cities_df.index[np.argmax(dists_from_top_city) + 1]
    all_indices.append(furthest_city_idx)
    
    # Now sample at random 50% from the remaining cities
    remaining_cities_ids = state_cities_df.index.to_list()
    [remaining_cities_ids.remove(x) for x in [top_city_idx, furthest_city_idx]]
    
    sampled_cities = np.random.choice(remaining_cities_ids, int(len(remaining_cities_ids) * 0.4), replace=False)
    [all_indices.append(x) for x in sampled_cities]

resampled_cities = cities_df.iloc[all_indices]

In [31]:
# Sanity check that we didn't take duplicate indices
print(len(all_indices))
print(len(np.unique(all_indices)))

201
201


In [32]:
# Last sanity check - control that indices in resampled_cities and indices in few_cities_df dont overlap
resampled_set = set(all_indices)
few_cities_set = set(few_cities_df.index.to_list())

print(len(set.intersection(resampled_set, few_cities_set)))

0


In [33]:
# Finally concatenate the two dataframes
final_cities_df = few_cities_df.append(resampled_cities, ignore_index=True)
final_cities_df

,state,city,population,geolocation
0,HI,Honolulu,953207,"(21.4588039305, -157.973296737)"
1,MD,Baltimore,620961,"(39.3084523991, -76.6160492311)"
2,DC,Washington,601723,"(38.9099241426, -77.0147205666)"
3,KY,Louisville,597337,"(38.1777689918, -85.6664099974)"
4,NV,Las Vegas,583756,"(36.2274148438, -115.262670095)"
...,...,...,...,...
244,LA,Shreveport,199311,"(32.4671618864, -93.7962236108)"
245,LA,Kenner,66702,"(30.0106937201, -90.2550320135)"
246,IA,Des Moines,203433,"(41.5741349525, -93.6165066086)"
247,IA,Sioux City,82684,"(42.4963149039, -96.3913834837)"


In [34]:
# Sanity check that we have 51 states

print(len(final_cities_df.state.unique()) == 51)

True


# NOW CONTINUE

### Read all counties

In [7]:
counties_df = pd.read_csv(coords_dir)
counties_df.head()
counties_df['geolocation'] = [eval(x) for x in counties_df['geolocation']]

In [9]:
# Sanity check there is at least one county for each of the above states

counties_df.shape

(3141, 5)

In [10]:
def find_top_county_per_city(final_cities_df, counties_df):
    
    num_cities = final_cities_df.shape[0]
    num_counties = counties_df.shape[0]
    
    # Populate dictionary mapping cities to best county 
    # Cities can have same name across states, so need to be more specific in the dictionary key
    city_to_county = {f"{final_cities_df.iloc[i]['state']}, {final_cities_df.iloc[i]['city']}":''
                      for i in range(num_cities)}
    
    # For each city compute distance from it to each county (lat, lon order)
    for i, city in enumerate(final_cities_df.city):

        dists = []
        city_coord = final_cities_df.iloc[i]['geolocation']
        
        # Only counties within the city's state can be used
        counties_state_df = counties_df[counties_df.state == final_cities_df.iloc[i]['state']]

        for j, county in enumerate(counties_state_df.county):
            county_coord = counties_state_df.iloc[j]['geolocation']
            dists.append(gd.distance(city_coord, county_coord).km)
        
        # There are many counties with the same name, need to track State as well
        best_idx = np.argmin(dists)
        city_to_county[f"{final_cities_df.iloc[i]['state']}, {final_cities_df.iloc[i]['city']}"] = \
        counties_state_df.iloc[best_idx][['fips_state', 'state']].to_numpy()
    
    return city_to_county

In [12]:
dc = find_top_county_per_city(cities_df, counties_df)

In [13]:
# Sanity: check we have one center per city
print(len(dc))

500


In [14]:
# Show an example
for k, v in dc.items():
    print(f"State: {v[1]}\nCity: {k}\nCounty: {v[0]}")
    break

State: NY
City: NY, New York
County: NY 36047


### Build final dataframe

In [17]:
colnames = ['state', 'city', 'fips', 'city_geolocation', 'county_geolocation', 'city_pop']
final = []
for city, county in dc.items():
    
    bare_city = city.split(',')[1].strip()

    county_arr = counties_df[(counties_df.fips_state==county[0]) &
                             (counties_df.state==county[1])
                            ][['state', 'geolocation']].to_numpy()[0]
    
    city_arr = cities_df[(cities_df.city == bare_city) &
                         (cities_df.state == county[1])
                        ][['geolocation', 'population']].to_numpy()[0]
    
    final.append([county_arr[0], bare_city, county[0], city_arr[0], county_arr[1], city_arr[1]])

complete_df = pd.DataFrame(final, columns=colnames).sort_values('city_pop', ascending=False)
complete_df.reset_index(drop=True, inplace=True)


In [18]:
complete_df

,state,city,fips,city_geolocation,county_geolocation,city_pop
0,NY,New York,NY 36047,"(40.694960689, -73.9313850409)","(40.6447, -73.9472)",8175133
1,CA,Los Angeles,CA 06037,"(34.1182277898, -118.408500088)","(34.3203, -118.2252)",3792621
2,IL,Chicago,IL 17031,"(41.8372950615, -87.6862308732)","(41.8399, -87.8167)",2695598
3,TX,Houston,TX 48201,"(29.7806691396, -95.3860033966)","(29.8588, -95.3963)",2099451
4,PA,Philadelphia,PA 42101,"(40.0093147808, -75.1333888571)","(40.0076, -75.1338)",1526006
...,...,...,...,...,...,...
495,SC,Rock Hill,SC 45091,"(34.9403611496, -81.0249178286)","(34.9747, -81.1845)",66154
496,NY,Schenectady,NY 36093,"(42.8025204414, -73.9275333869)","(42.8182, -74.0585)",66135
497,WY,Cheyenne,WY 56021,"(41.1460804265, -104.789064332)","(41.3071, -104.6896)",59466
498,WV,Charleston,WV 54039,"(38.3484079736, -81.632165003)","(38.3365, -81.5281)",51400


### Build matrix of distances between counties

In [19]:
# Pre-populate array (rows are the assigned centers, columns are each existing county)
counties_dists = np.ones(shape=(complete_df.shape[0], counties_df.shape[0])) * -1.0

for i in range(complete_df.shape[0]):
    
    # Compute distance from selected county to all other counties within the same state
    dists = []
    center_coord = complete_df.iloc[i]['county_geolocation']

    counties_state_df = counties_df[counties_df.state == complete_df.iloc[i]['state']]
    nonzero_indices = counties_state_df.index
    
    for j, county in enumerate(counties_state_df.county):
        county_coord = counties_state_df.iloc[j]['geolocation']
        dists.append(gd.distance(center_coord, county_coord).km)
    
    counties_dists[i, nonzero_indices] = dists

In [20]:
columns = counties_df.fips_state.to_numpy()
index = complete_df['state'] + ', ' + complete_df['city']

counties_dists_df = pd.DataFrame(counties_dists, 
                                 columns=columns, 
                                 index=index)

In [22]:
counties_dists_df.shape

(500, 3141)

In [25]:
# One more sanity check (check distance of center with itself is 0)
example = counties_dists_df.loc["NY, New York"]
example[example > -1].head(40)

NY 36001    217.200563
NY 36003    385.075581
NY 36005     24.073172
NY 36007    229.867055
NY 36009    433.591751
NY 36011    332.630171
NY 36013    485.756364
NY 36015    288.013523
NY 36017    247.852975
NY 36019    456.155176
NY 36021    180.236858
NY 36023    279.669403
NY 36025    192.421726
NY 36027    125.615610
NY 36029    462.316462
NY 36031    385.995127
NY 36033    439.560546
NY 36035    277.074636
NY 36037    439.170765
NY 36039    181.835424
NY 36041    338.120047
NY 36043    319.475060
NY 36045    410.433725
NY 36047      0.000000
NY 36049    370.147874
NY 36051    393.787721
NY 36053    289.766382
NY 36055    417.029917
NY 36057    254.069221
NY 36059     32.153276
NY 36061     14.982008
NY 36063    488.600015
NY 36065    313.756043
NY 36067    321.891936
NY 36069    371.309602
NY 36071     89.360904
NY 36073    458.213838
NY 36075    358.365604
NY 36077    238.708284
NY 36079     88.421507
Name: NY, New York, dtype: float64

In [26]:
sum(sum(counties_dists>-1))

41505

### Save all these big boys

In [27]:
complete_df.to_csv('selected_centers_500.csv')
counties_dists_df.to_csv('counties_distances_500.csv')